<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Evaluate RAG with LLM Evals</h1>

In this tutorial we will look into building a RAG pipeline and evaluating it with Phoenix Evals.

It has the the following sections:

1. Understanding Retrieval Augmented Generation (RAG).
2. Building RAG (with the help of a framework such as LlamaIndex).
3. Evaluating RAG with Phoenix Evals.

## Retrieval Augmented Generation (RAG)

LLMs are trained on vast datasets, but these will not include your specific data (things like company knowledge bases and documentation). Retrieval-Augmented Generation (RAG) addresses this by dynamically incorporating your data as context during the generation process. This is done not by altering the training data of the LLMs but by allowing the model to access and utilize your data in real-time to provide more tailored and contextually relevant responses.

In RAG, your data is loaded and prepared for queries. This process is called indexing. User queries act on this index, which filters your data down to the most relevant context. This context and your query then are sent to the LLM along with a prompt, and the LLM provides a response.

RAG is a critical component for building applications such a chatbots or agents and you will want to know RAG techniques on how to get data into your application.

<img src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/RAG_Pipeline.png">

## Stages within RAG

There are five key stages within RAG, which will in turn be a part of any larger RAG application.

- **Loading**: This refers to getting your data from where it lives - whether it's text files, PDFs, another website, a database or an API - into your pipeline.
- **Indexing**: This means creating a data structure that allows for querying the data. For LLMs this nearly always means creating vector embeddings, numerical representations of the meaning of your data, as well as numerous other metadata strategies to make it easy to accurately find contextually relevant data.
- **Storing**: Once your data is indexed, you will want to store your index, along with any other metadata, to avoid the need to re-index it.

- **Querying**: For any given indexing strategy there are many ways you can utilize LLMs and data structures to query, including sub-queries, multi-step queries, and hybrid strategies. 
- **Evaluation**: A critical step in any pipeline is checking how effective it is relative to other strategies, or when you make changes. Evaluation provides objective measures on how accurate, faithful, and fast your responses to queries are.


## Build a RAG system 

Now that we have understood the stages of RAG, let's build a pipeline. We will use [LlamaIndex](https://www.llamaindex.ai/) for RAG and [Phoenix Evals](https://docs.arize.com/phoenix/llm-evals/llm-evals) for evaluation.


In [1]:
%pip install -q -U llama-index-core llama-index-readers-file llama-index-llms-openai llama-index-embeddings-openai arize-phoenix


Note: you may need to restart the kernel to use updated packages.


For this tutorial we will be using OpenAI for creating synthetic data as well as for evaluation. 

In [2]:
import os
import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
openai_client = OpenAI()

In [3]:
# The nest_asyncio module enables the nesting of asynchronous functions within an already running async loop.
# This is necessary because Jupyter notebooks inherently operate in an asynchronous loop.
# By applying nest_asyncio, we can run additional async functions within this existing loop without conflicts.
import nest_asyncio

nest_asyncio.apply()

import pandas as pd
import phoenix as px
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.llms.openai import OpenAI

During this tutorial, we will capture all the data we need to evaluate our RAG pipeline using Phoenix Tracing. To enable this, simply start the phoenix application and instrument LlamaIndex.

In [4]:
px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


Enable Phoenix tracing via `LlamaIndexInstrumentor`. Phoenix uses OpenInference traces - an open-source standard for capturing and storing LLM application traces that enables LLM applications to seamlessly integrate with LLM observability solutions such as Phoenix.

In [5]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

### Load Data and Build an Index

Let's use an [essay by Paul Graham](https://www.paulgraham.com/worked.html) to build our RAG pipeline.

In [6]:
import tempfile
from urllib.request import urlretrieve

with tempfile.NamedTemporaryFile() as tf:
    urlretrieve(
        "https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/data/paul_graham/paul_graham_essay.txt",
        tf.name,
    )
    documents = SimpleDirectoryReader(input_files=[tf.name]).load_data()

In [7]:
# Define an LLM
llm = OpenAI(model="gpt-4o")

# Build index with a chunk_size of 512
node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)
vector_index = VectorStoreIndex(nodes)

Build a QueryEngine and start querying.

In [8]:
query_engine = vector_index.as_query_engine()

In [9]:
response_vector = query_engine.query("What did the author do growing up?")

Check the response that you get from the query.

In [10]:
response_vector.response

'The author worked on writing short stories and programming, particularly on an IBM 1401 computer using an early version of Fortran during 9th grade.'

By default LlamaIndex retrieves two similar nodes/ chunks. You can modify that in `vector_index.as_query_engine(similarity_top_k=k)`.

Let's check the text in each of these retrieved nodes.

In [11]:
# First retrieved node
response_vector.source_nodes[0].get_text()

'What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then stack

In [12]:
# Second retrieved node
response_vector.source_nodes[1].get_text()

"Except for a few officially anointed thinkers who went to the right parties in New York, the only people allowed to publish essays were specialists writing about their specialties. There were so many essays that had never been written, because there had been no way to publish them. Now they could be, and I was going to write them. [12]\n\nI've worked on several different things, but to the extent there was a turning point where I figured out what to work on, it was when I started publishing essays online. From then on I knew that whatever else I did, I'd always write essays too.\n\nI knew that online essays would be a marginal medium at first. Socially they'd seem more like rants posted by nutjobs on their GeoCities sites than the genteel and beautifully typeset compositions published in The New Yorker. But by this point I knew enough to find that encouraging instead of discouraging.\n\nOne of the most conspicuous patterns I've noticed in my life is how well it has worked, for me at l

Remember that we are using Phoenix Tracing to capture all the data we need to evaluate our RAG pipeline. You can view the traces in the phoenix application.

In [13]:
print("phoenix URL", px.active_session().url)

phoenix URL http://localhost:6006/


We can access the traces by directly pulling the spans from the phoenix session.

In [14]:
spans_df = px.Client().get_spans_dataframe()

/home/vscode/my-envs/03-rag-graph-databases/lib/python3.12/site-packages/phoenix/trace/dsl/query.py:746: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_attributes = pd.DataFrame.from_records(
/home/vscode/my-envs/03-rag-graph-databases/lib/python3.12/site-packages/phoenix/trace/dsl/query.py:746: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_attributes = pd.DataFrame.from_records(
/home/vscode/my-envs/03-rag-graph-databases/lib/python3.12/site-packages/phoenix/trace/dsl/query.py:746: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treate

In [15]:
spans_df[["name", "span_kind", "attributes.input.value", "attributes.retrieval.documents"]].head()

,name,span_kind,attributes.input.value,attributes.retrieval.documents
context.span_id,,,,
088d92f479f2c0d7,SentenceSplitter.split_text_metadata_aware,CHAIN,"{""text"": ""\n\nWhat I Worked On\n\nFebruary 202...",None
a1e9396470e3b189,MetadataAwareTextSplitter._parse_nodes,CHAIN,"{""nodes"": [""<Document(id_=e4a9b8ea-7019-4021-9...",None
101d771963ad3efd,BaseEmbedding.get_text_embedding_batch,EMBEDDING,"{""texts"": [""file_path: /tmp/tmp1g6e0hf2\n\nWha...",None
775fc27644977b7b,OpenAIEmbedding._get_query_embedding,EMBEDDING,"{""query"": ""What did the author do growing up?""}",None
f90edd8971531174,BaseEmbedding.get_query_embedding,EMBEDDING,"{""query"": ""What did the author do growing up?""}",None


Note that the traces have captured the documents that were retrieved by the query engine. This is nice because it means we can introspect the documents without having to keep track of them ourselves.

In [16]:
spans_with_docs_df = spans_df[spans_df["attributes.retrieval.documents"].notnull()]

In [17]:
spans_with_docs_df[["attributes.input.value", "attributes.retrieval.documents"]].head()

,attributes.input.value,attributes.retrieval.documents
context.span_id,,
4d6951a614a47997,What did the author do growing up?,[{'document.content': 'What I Worked On Febru...


We have built a RAG pipeline and also have instrumented it using Phoenix Tracing. We now need to evaluate it's performance. We can assess our RAG system/query engine using Phoenix's LLM Evals. Let's examine how to leverage these tools to quantify the quality of our retrieval-augmented generation system.

## Evaluation

Evaluation should serve as the primary metric for assessing your RAG application. It determines whether the pipeline will produce accurate responses based on the data sources and range of queries.

While it's beneficial to examine individual queries and responses, this approach is impractical as the volume of edge-cases and failures increases. Instead, it's more effective to establish a suite of metrics and automated evaluations. These tools can provide insights into overall system performance and can identify specific areas that may require scrutiny.

In a RAG system, evaluation focuses on two critical aspects:

- **Retrieval Evaluation**: To assess the accuracy and relevance of the documents that were retrieved
- **Response Evaluation**: Measure the appropriateness of the response generated by the system when the context was provided.

### Generate Question Context Pairs

For the evaluation of a RAG system, it's essential to have queries that can fetch the correct context and subsequently generate an appropriate response.

For this tutorial, let's use Phoenix's `llm_generate` to help us create the question-context pairs.

First, let's create a dataframe of all the document chunks that we have indexed.

In [18]:
# Let's construct a dataframe of just the documents that are in our index
document_chunks_df = pd.DataFrame({"text": [node.get_text() for node in nodes]})
document_chunks_df = document_chunks_df.sample(10, random_state=42)
document_chunks_df.head()

,text
0,What I Worked On\n\nFebruary 2021\n\nBefore co...
5,"It was not, in fact, simply a matter of teachi..."
30,It was exciting for a while. Painting started ...
13,"I wanted to go back to RISD, but I was now bro..."
34,Except for a few officially anointed thinkers ...


Now that we have the document chunks, let's prompt an LLM to generate us 3 questions per chunk. Note that you could manually solicit questions from your team or customers, but this is a quick and easy way to generate a large number of questions.

In [19]:
generate_questions_template = """\
Context information is below.

---------------------
{text}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Teacher/ Professor. Your task is to setup \
3 questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. Restrict the questions to the \
context information provided."

Output the questions in JSON format with the keys question_1, question_2, question_3.
"""

In [20]:
import json

from phoenix.evals import OpenAIModel, llm_generate


def output_parser(response: str, index: int):
    try:
        return json.loads(response)
    except json.JSONDecodeError as e:
        return {"__error__": str(e)}


questions_df = llm_generate(
    dataframe=document_chunks_df,
    template=generate_questions_template,
    model=OpenAIModel(model="gpt-3.5-turbo"),
    output_parser=output_parser,
    concurrency=20,
)

llm_generate |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

In [21]:
questions_df.head()

,question_1,question_2,question_3
0,What were the two main things the author worke...,Describe the author's experience with programm...,How did the author's experience with microcomp...
5,"What was the author's initial approach to AI, ...",Why did the author decide to focus on Lisp and...,What was the author's dilemma between theory a...
30,What new approach did the author experiment wi...,What idea did the author have in the spring of...,Why did the author's friend Robert refuse to w...
13,What was the job the author took at Interleaf ...,How did the author manage to save enough money...,What were some of the challenges the author fa...
34,What was the turning point for the author in f...,Why does the author believe that working on un...,What were some of the different topics the aut...


In [22]:
# Construct a dataframe of the questions and the document chunks
questions_with_document_chunk_df = pd.concat([questions_df, document_chunks_df], axis=1)
questions_with_document_chunk_df = questions_with_document_chunk_df.melt(
    id_vars=["text"], value_name="question"
).drop("variable", axis=1)
# If the above step was interrupted, there might be questions missing. Let's run this to clean up the dataframe.
questions_with_document_chunk_df = questions_with_document_chunk_df[
    questions_with_document_chunk_df["question"].notnull()
]

The LLM has generated three questions per chunk. Let's take a quick look.

In [23]:
questions_with_document_chunk_df.head(10)

,text,question
0,What I Worked On\n\nFebruary 2021\n\nBefore co...,What were the two main things the author worke...
1,"It was not, in fact, simply a matter of teachi...","What was the author's initial approach to AI, ..."
2,It was exciting for a while. Painting started ...,What new approach did the author experiment wi...
3,"I wanted to go back to RISD, but I was now bro...",What was the job the author took at Interleaf ...
4,Except for a few officially anointed thinkers ...,What was the turning point for the author in f...
5,[14] YC did become a fund for a couple years s...,Explain the significance of YC becoming a fund...
6,In principle our Viaweb stock was valuable. It...,How did the author feel about the value of the...
7,I hoped to lure Robert into working on it with...,What was the initial reason for starting Viawe...
8,"And it wasn't, so I would.\n\nIn the summer of...",What was the reason behind the decision to han...
9,[4]\n\nI liked painting still lives because I ...,How did the author's experience with painting ...


### Retrieval Evaluation

We are now prepared to perform our retrieval evaluations. We will execute the queries we generated in the previous step and verify whether or not that the correct context is retrieved.

In [24]:
# loop over the questions and generate the answers
for _, row in questions_with_document_chunk_df.iterrows():
    question = row["question"]
    response_vector = query_engine.query(question)
    print(f"Question: {question}\nAnswer: {response_vector.response}\n")

Question: What were the two main things the author worked on before college?
Answer: The author worked on writing and programming before college.

Question: What was the author's initial approach to AI, and why did they realize it was not going to work?
Answer: The author's initial approach to AI involved using programs that utilized explicit data structures to represent concepts based on natural language input. However, the author realized that this approach was not going to work because it only demonstrated a subset of natural language as a formal language, showing an unbridgeable gap between what the programs could do and actual understanding of natural language. The author concluded that teaching these programs more words or applying band-aids would not solve the fundamental issue with this approach to AI.

Question: What new approach did the author experiment with in their painting, involving using a printed canvas as an underpainting for a second still life?
Answer: The author ex

Now that we have executed the queries, we can start validating whether or not the RAG system was able to retrieve the correct context. Let's extract all the retrieved documents from the traces logged to phoenix. (For an in-depth explanation of how to export trace data from the phoenix runtime, consult the [docs](https://docs.arize.com/phoenix/how-to/extract-data-from-spans)).

In [25]:
from phoenix.session.evaluation import get_retrieved_documents

retrieved_documents_df = get_retrieved_documents(px.Client())
retrieved_documents_df

context.trace_id  \
context.span_id  document_position                                     
4d6951a614a47997 0                  ffc4460b03df8cb3cc6c903e51fa3eaf   
                 1                  ffc4460b03df8cb3cc6c903e51fa3eaf   
1b5a2b502a7dc807 0                  f96c86778fa5c5d075eab4730ece01f5   
                 1                  f96c86778fa5c5d075eab4730ece01f5   
c386eb79c1700d1a 0                  fe39aa2e7cc75264654b8f64abc45dec   
...                                                              ...   
a7e6b4488559055c 1                  fe8b8c2ebd46e40ca4506ea918c20eb3   
ad4a55bc3c1c495b 0                  5d401b5c9e2cd022103374b2bb8dbf80   
                 1                  5d401b5c9e2cd022103374b2bb8dbf80   
69567c1656bdca76 0                  d8c1082820a2edac9cf2bc24f63777b7   
                 1                  d8c1082820a2edac9cf2bc24f63777b7   

                                                                                input  \
context.span_id  document_position                                                      
4d6951a614a47997 0                                 What did the author do growing up?   
                 1                                 What did the author do growing up?   
1b5a2b502a7dc807 0                  What were the two main things the author worke...   
                 1                  What were the two main things the author worke...   
c386eb79c1700d1a 0                  What was the author's initial approach to AI, ...   
...                                                                               ...   
a7e6b4488559055c 1                  What was the name of the new company the autho...   
ad4a55bc3c1c495b 0                  After the passing of the author's mother, what...   
                 1                  After the passing of the author's mother, what...   
69567c1656bdca76 0                  What unique feature did Interleaf software hav...   
                 1                  What unique feature did Interleaf software hav...   

                                                                            reference  \
context.span_id  document_position                                                      
4d6951a614a47997 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  Except for a few officially anointed thinkers ...   
1b5a2b502a7dc807 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  Except for a few officially anointed thinkers ...   
c386eb79c1700d1a 0                  I had gotten into a program at Cornell that di...   
...                                                                               ...   
a7e6b4488559055c 1                  But about halfway through the summer I realize...   
ad4a55bc3c1c495b 0                  And it wasn't, so I would.\n\nIn the summer of...   
                 1                  Now when I talk to founders who are leaving af...   
69567c1656bdca76 0                  I wanted to go back to RISD, but I was now bro...   
                 1                  That seemed unnatural to me, and on this point...   

                                    document_score  
context.span_id  document_position                  
4d6951a614a47997 0                        0.813484  
                 1                        0.811437  
1b5a2b502a7dc807 0                        0.838987  
                 1                        0.834713  
c386eb79c1700d1a 0                        0.864163  
...                                            ...  
a7e6b4488559055c 1                        0.825835  
ad4a55bc3c1c495b 0                        0.828666  
                 1                        0.815216  
69567c1656bdca76 0                        0.862923  
                 1                        0.840454  

[62 rows x 4 columns]

Let's now use Phoenix's LLM Evals to evaluate the relevance of the retrieved documents with regards to the query. Note, we've turned on `explanations` which prompts the LLM to explain it's reasoning. This can be useful for debugging and for figuring out potential corrective actions.

In [26]:
from phoenix.evals import (
    RelevanceEvaluator,
    run_evals,
)

relevance_evaluator = RelevanceEvaluator(OpenAIModel(model="gpt-4o"))

retrieved_documents_relevance_df = run_evals(
    evaluators=[relevance_evaluator],
    dataframe=retrieved_documents_df,
    provide_explanation=True,
    concurrency=20,
)[0]

run_evals |          | 0/62 (0.0%) | ⏳ 00:00<? | ?it/s

In [27]:
retrieved_documents_relevance_df.head()

label  score  \
context.span_id  document_position                        
4d6951a614a47997 0                      relevant    1.0   
                 1                     unrelated    0.0   
1b5a2b502a7dc807 0                      relevant    1.0   
                 1                     unrelated    0.0   
c386eb79c1700d1a 0                  NOT_PARSABLE    0.0   

                                                                          explanation  
context.span_id  document_position                                                     
4d6951a614a47997 0                  To determine if the reference text is relevant...  
                 1                  To determine if the reference text is relevant...  
1b5a2b502a7dc807 0                  To determine if the reference text is relevant...  
                 1                  To determine if the reference text contains in...  
c386eb79c1700d1a 0                  To determine if the reference text is relevant...

We can now combine the documents with the relevance evaluations to compute retrieval metrics. These metrics will help us understand how well the RAG system is performing.

In [28]:
documents_with_relevance_df = pd.concat(
    [retrieved_documents_df, retrieved_documents_relevance_df.add_prefix("eval_")], axis=1
)
documents_with_relevance_df

context.trace_id  \
context.span_id  document_position                                     
4d6951a614a47997 0                  ffc4460b03df8cb3cc6c903e51fa3eaf   
                 1                  ffc4460b03df8cb3cc6c903e51fa3eaf   
1b5a2b502a7dc807 0                  f96c86778fa5c5d075eab4730ece01f5   
                 1                  f96c86778fa5c5d075eab4730ece01f5   
c386eb79c1700d1a 0                  fe39aa2e7cc75264654b8f64abc45dec   
...                                                              ...   
a7e6b4488559055c 1                  fe8b8c2ebd46e40ca4506ea918c20eb3   
ad4a55bc3c1c495b 0                  5d401b5c9e2cd022103374b2bb8dbf80   
                 1                  5d401b5c9e2cd022103374b2bb8dbf80   
69567c1656bdca76 0                  d8c1082820a2edac9cf2bc24f63777b7   
                 1                  d8c1082820a2edac9cf2bc24f63777b7   

                                                                                input  \
context.span_id  document_position                                                      
4d6951a614a47997 0                                 What did the author do growing up?   
                 1                                 What did the author do growing up?   
1b5a2b502a7dc807 0                  What were the two main things the author worke...   
                 1                  What were the two main things the author worke...   
c386eb79c1700d1a 0                  What was the author's initial approach to AI, ...   
...                                                                               ...   
a7e6b4488559055c 1                  What was the name of the new company the autho...   
ad4a55bc3c1c495b 0                  After the passing of the author's mother, what...   
                 1                  After the passing of the author's mother, what...   
69567c1656bdca76 0                  What unique feature did Interleaf software hav...   
                 1                  What unique feature did Interleaf software hav...   

                                                                            reference  \
context.span_id  document_position                                                      
4d6951a614a47997 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  Except for a few officially anointed thinkers ...   
1b5a2b502a7dc807 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  Except for a few officially anointed thinkers ...   
c386eb79c1700d1a 0                  I had gotten into a program at Cornell that di...   
...                                                                               ...   
a7e6b4488559055c 1                  But about halfway through the summer I realize...   
ad4a55bc3c1c495b 0                  And it wasn't, so I would.\n\nIn the summer of...   
                 1                  Now when I talk to founders who are leaving af...   
69567c1656bdca76 0                  I wanted to go back to RISD, but I was now bro...   
                 1                  That seemed unnatural to me, and on this point...   

                                    document_score    eval_label  eval_score  \
context.span_id  document_position                                             
4d6951a614a47997 0                        0.813484      relevant         1.0   
                 1                        0.811437     unrelated         0.0   
1b5a2b502a7dc807 0                        0.838987      relevant         1.0   
                 1                        0.834713     unrelated         0.0   
c386eb79c1700d1a 0                        0.864163  NOT_PARSABLE         0.0   
...                                            ...           ...         ...   
a7e6b4488559055c 1                        0.825835     unrelated         0.0   
ad4a55bc3c1c495b 0                        0.828666      relevant         1.0   

Let's compute Normalized Discounted Cumulative Gain [NCDG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) at 2 for all our retrieval steps.  In information retrieval, this metric is often used to measure effectiveness of search engine algorithms and related applications.

In [29]:
import numpy as np
from sklearn.metrics import ndcg_score


def _compute_ndcg(df: pd.DataFrame, k: int):
    """Compute NDCG@k in the presence of missing values"""
    n = max(2, len(df))
    eval_scores = np.zeros(n)
    doc_scores = np.zeros(n)
    eval_scores[: len(df)] = df.eval_score
    doc_scores[: len(df)] = df.document_score
    try:
        return ndcg_score([eval_scores], [doc_scores], k=k)
    except ValueError:
        return np.nan


ndcg_at_2 = pd.DataFrame(
    {"score": documents_with_relevance_df.groupby("context.span_id").apply(_compute_ndcg, k=2)}
)

In [30]:
ndcg_at_2

,score
context.span_id,
003eea083158db74,1.00000
15bb213f1cfd1d7e,1.00000
1b5a2b502a7dc807,1.00000
1b9c2a6c5c5e51e3,1.00000
24050f056930b408,1.00000
24d5e8cc0b6b1d4c,1.00000
2c847fe847a95408,1.00000
2f74ab711ba7f268,1.00000
3a685930062c2602,1.00000


Let's also compute precision at 2 for all our retrieval steps.

In [31]:
precision_at_2 = pd.DataFrame(
    {
        "score": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) / 2
        )
    }
)

In [32]:
precision_at_2

,score
context.span_id,
003eea083158db74,1.0
15bb213f1cfd1d7e,1.0
1b5a2b502a7dc807,0.5
1b9c2a6c5c5e51e3,1.0
24050f056930b408,0.5
24d5e8cc0b6b1d4c,0.5
2c847fe847a95408,0.5
2f74ab711ba7f268,0.5
3a685930062c2602,1.0


Lastly, let's compute whether or not a correct document was retrieved at all for each query (e.g. a hit)

In [33]:
hit = pd.DataFrame(
    {
        "hit": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) > 0
        )
    }
)

Let's now view the results in a combined dataframe.

In [35]:
retrievals_df = px.Client().get_spans_dataframe(
    "span_kind == 'RETRIEVER' and input.value is not None"
)
rag_evaluation_dataframe = pd.concat(
    [
        retrievals_df["attributes.input.value"],
        ndcg_at_2.add_prefix("ncdg@2_"),
        precision_at_2.add_prefix("precision@2_"),
        hit,
    ],
    axis=1,
)
rag_evaluation_dataframe

,attributes.input.value,ncdg@2_score,precision@2_score,hit
context.span_id,,,,
5106ae6a74c9cc93,"{""query_bundle"": {""query_str"": ""What did the a...",NaN,NaN,NaN
4d6951a614a47997,What did the author do growing up?,1.0,0.5,True
72a9a2fd68404457,"{""query_bundle"": {""query_str"": ""What were the ...",NaN,NaN,NaN
1b5a2b502a7dc807,What were the two main things the author worke...,1.0,0.5,True
06204ae05a56b478,"{""query_bundle"": {""query_str"": ""What was the a...",NaN,NaN,NaN
...,...,...,...,...
a7e6b4488559055c,What was the name of the new company the autho...,0.0,0.0,False
8cc3ef9389f08fe9,"{""query_bundle"": {""query_str"": ""After the pass...",NaN,NaN,NaN
ad4a55bc3c1c495b,"After the passing of the author's mother, what...",1.0,0.5,True


### Observations

Let's now take our results and aggregate them to get a sense of how well our RAG system is performing.

In [36]:
# Aggregate the scores across the retrievals
results = rag_evaluation_dataframe.mean(numeric_only=True)
results

ncdg@2_score         0.859062
precision@2_score    0.693548
dtype: float64

As we can see from the above numbers, our RAG system is not perfect, there are times when it fails to retrieve the correct context within the first two documents. At other times the correct context is included in the top 2 results but non-relevant information is also included in the context. This is an indication that we need to improve our retrieval strategy. One possible solution could be to increase the number of documents retrieved and then use a more sophisticated ranking strategy (such as a reranker) to select the correct context.

We have now evaluated our RAG system's retrieval performance. Let's send these evaluations to Phoenix for visualization. By sending the evaluations to Phoenix, you will be able to view the evaluations alongside the traces that were captured earlier.

In [37]:
from phoenix.trace import DocumentEvaluations, SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=ndcg_at_2, eval_name="ndcg@2"),
    SpanEvaluations(dataframe=precision_at_2, eval_name="precision@2"),
    DocumentEvaluations(dataframe=retrieved_documents_relevance_df, eval_name="relevance"),
)

### Response Evaluation

The retrieval evaluations demonstrates that our RAG system is not perfect. However, it's possible that the LLM is able to generate the correct response even when the context is incorrect. Let's evaluate the responses generated by the LLM.

In [38]:
from phoenix.session.evaluation import get_qa_with_reference

qa_with_reference_df = get_qa_with_reference(px.Client())
qa_with_reference_df

,input,output,reference
context.span_id,,,
7ac12534d2e9b50a,What did the author do growing up?,The author worked on writing short stories and...,What I Worked On\n\nFebruary 2021\n\nBefore co...
962b5e481a71bdc8,What were the two main things the author worke...,The author worked on writing and programming b...,What I Worked On\n\nFebruary 2021\n\nBefore co...
26de561982bac687,"What was the author's initial approach to AI, ...",The author's initial approach to AI involved u...,I had gotten into a program at Cornell that di...
49e609dee4cd79d4,What new approach did the author experiment wi...,The author experimented with a new approach in...,I remember that I answered the essay question ...
37154901e115d853,What was the job the author took at Interleaf ...,The job the author took at Interleaf was to wr...,"I wanted to go back to RISD, but I was now bro..."
ffc657681e1a7769,What was the turning point for the author in f...,The turning point for the author in figuring o...,Except for a few officially anointed thinkers ...
f5555d489d00fb22,Explain the significance of YC becoming a fund...,The significance of YC becoming a fund in 2009...,"We had no idea how to be angel investors, and ..."
ea87d9a1e7bdba01,How did the author feel about the value of the...,The author did not feel that their Viaweb stoc...,In principle our Viaweb stock was valuable. It...
4e522309318b2a4c,What was the initial reason for starting Viawe...,The initial reason for starting Viaweb accordi...,But about halfway through the summer I realize...


Now that we have a dataset of the question, context, and response (input, reference, and output), we now can measure how well the LLM is responding to the queries. For details on the QA correctness evaluation, see the [LLM Evals documentation](https://docs.arize.com/phoenix/llm-evals/running-pre-tested-evals/q-and-a-on-retrieved-data).

In [39]:
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    run_evals,
)

qa_evaluator = QAEvaluator(OpenAIModel(model="gpt-4-turbo-preview"))
hallucination_evaluator = HallucinationEvaluator(OpenAIModel(model="gpt-4-turbo-preview"))

qa_correctness_eval_df, hallucination_eval_df = run_evals(
    evaluators=[qa_evaluator, hallucination_evaluator],
    dataframe=qa_with_reference_df,
    provide_explanation=True,
    concurrency=20,
)

run_evals |          | 0/62 (0.0%) | ⏳ 00:00<? | ?it/s

In [40]:
qa_correctness_eval_df.head()

,label,score,explanation
context.span_id,,,
7ac12534d2e9b50a,correct,1,The given answer accurately reflects the activ...
962b5e481a71bdc8,correct,1,"To determine if the answer is correct, we comp..."
26de561982bac687,correct,1,The given answer accurately reflects the autho...
49e609dee4cd79d4,incorrect,0,The reference text does not mention the author...
37154901e115d853,correct,1,The given answer accurately reflects the infor...


In [41]:
hallucination_eval_df.head()

,label,score,explanation
context.span_id,,,
7ac12534d2e9b50a,factual,0,The answer is factual because it accurately re...
962b5e481a71bdc8,factual,0,The reference text explicitly states that befo...
26de561982bac687,factual,0,The answer accurately reflects the content and...
49e609dee4cd79d4,hallucinated,1,The reference text does not mention the author...
37154901e115d853,factual,0,The answer accurately reflects the information...


#### Observations

Let's now take our results and aggregate them to get a sense of how well the LLM is answering the questions given the context.

In [42]:
qa_correctness_eval_df.mean(numeric_only=True)

score    0.870968
dtype: float64

In [43]:
hallucination_eval_df.mean(numeric_only=True)

score    0.096774
dtype: float64

Our QA Correctness score of `0.91` and a Hallucinations score `0.05` signifies that the generated answers are correct ~91% of the time and that the responses contain hallucinations 5% of the time - there is room for improvement. This could be due to the retrieval strategy or the LLM itself. We will need to investigate further to determine the root cause.

Since we have evaluated our RAG system's QA performance and Hallucinations performance, let's send these evaluations to Phoenix for visualization.

In [44]:
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=qa_correctness_eval_df, eval_name="Q&A Correctness"),
    SpanEvaluations(dataframe=hallucination_eval_df, eval_name="Hallucination"),
)

We now have sent all our evaluations to Phoenix. Let's go to the Phoenix application and view the results! Since we've sent all the evals to Phoenix, we can analyze the results together to make a determination on whether or not poor retrieval or irrelevant context has an effect on the LLM's ability to generate the correct response.

In [45]:
print("phoenix URL", px.active_session().url)

phoenix URL http://localhost:6006/


: 

## Conclusion

We have explored how to build and evaluate a RAG pipeline using LlamaIndex and Phoenix, with a specific focus on evaluating the retrieval system and generated responses within the pipelines. 

Phoenix offers a variety of other evaluations that can be used to assess the performance of your LLM Application. For more details, see the [LLM Evals](https://docs.arize.com/phoenix/llm-evals/llm-evals) documentation.